In [ ]:
import sys

sys.path.insert(0, "..")
sys.path.insert(0, "../disc_cop")
from disc_cop.envs import ENV_ID_TO_NAME

import _pickle as pickle
import matplotlib.pyplot as plt
import numpy as np
import os
import seaborn as sns
import torch
import pandas as pd

from tqdm.notebook import tqdm

In [ ]:
result_dir = "/Volumes/Expansion/icml_2025/avg_ope/cc_results/results/saved_models"
stats_file = "/Volumes/Expansion/icml_2025/avg_ope/cc_results/results/processed_best.feather"

In [ ]:
stats = []

for env_name in ENV_ID_TO_NAME.values():
    if not os.path.isdir(os.path.join(result_dir, env_name)):
        print("{} not processed".format(env_name))
        continue

    for run_file in tqdm(os.listdir(os.path.join(result_dir, env_name)), desc=env_name):
        if run_file.startswith("."):
            continue

        prefix = run_file.split("-")[:4]

        for seed in range(10):
            torch_path = os.path.join(result_dir, env_name, run_file, "seed_{}-curr_best.pt".format(seed))
            data = torch.load(torch_path)
            curr_loss = data["loss"]
            curr_steps = data["steps"]

            stats.append(
                dict(
                    variant=run_file,
                    env_name=env_name,
                    seed=seed,
                    **{
                        "_".join(key_val.split("_")[:-1]): key_val.split("_")[-1] for key_val in prefix
                    },
                    loss=curr_loss,
                    steps=curr_steps,
                )
            )

In [ ]:
stats = pd.DataFrame(stats)
stats.to_feather(stats_file)

In [ ]:
stats